## Project Introduction: Document Classification with Machine Learning

In this personal project, I explore how machine learning can be used to automatically classify news articles into relevant categories. This kind of document classification is a key component of intelligent content search engines, helping users find information quickly and accurately.

To implement this, I’ll use **PyTorch** and the **torchtext** library to preprocess and prepare raw text data for model training. The pipeline includes converting text into tensors, organizing it into batches, and building a classifier capable of predicting the topic of a given article.

This project demonstrates the practical application of natural language processing (NLP) and deep learning in organizing large volumes of unstructured text—a valuable capability for search, recommendation, and information retrieval systems.


In [ ]:

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchtext.data.utils import get_tokenizer
from collections import Counter
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import pickle
import os

In [ ]:
torch.manual_seed(42)
np.random.seed(42)

class NewsDataset(Dataset):
    """Custom Dataset class for news articles"""
    
    def __init__(self, texts, labels, vocab, tokenizer, max_length=512):
        self.texts = texts
        self.labels = labels
        self.vocab = vocab
        self.tokenizer = tokenizer
        self.max_length = max_length
        
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        text = str(self.texts[idx])
        label = self.labels[idx]
        
        # Tokenize and convert to indices
        tokens = self.tokenizer(text.lower())
        indices = [self.vocab.get(token, self.vocab['<UNK>']) for token in tokens]
        
        # Pad or truncate to max_length
        if len(indices) > self.max_length:
            indices = indices[:self.max_length]
        else:
            indices.extend([self.vocab['<PAD>']] * (self.max_length - len(indices)))
            
        return torch.tensor(indices, dtype=torch.long), torch.tensor(label, dtype=torch.long)